In [1]:
import pandas as pd
import numpy as np
import os
import json

# Dataset-b 다운로드

In [ ]:
!gdown https://drive.google.com/uc?id=1pIQxkhetz9o2n5DsebKQBz0EwM8-IefV
!unzip ./cocoset_b.zip
!rm ./cocoset_b.zip

In [3]:
with open('/content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/annotations/instances_train2017.json', 'r') as f:
    train_json = json.load(f)

with open('/content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/annotations/instances_val2017.json', 'r') as f:
    val_json = json.load(f)    

In [5]:
train_json.keys()

dict_keys(['info', 'images', 'licenses', 'type', 'annotations', 'categories'])

In [18]:
train_json['images'][-1]

{'date_captured': '2021',
 'file_name': '000000012304.jpg',
 'id': 12304,
 'height': 512,
 'width': 512}

In [19]:
train_json['annotations'][-1]

{'segmentation': [[172.0, 148.0, 228.0, 148.0, 228.0, 204.0, 172.0, 204.0]],
 'area': 3136.0,
 'iscrowd': 0,
 'image_id': 12304,
 'bbox': [172.0, 148.0, 56.0, 56.0],
 'category_id': 1,
 'id': 101140}

train_json['images']의 'id'와 train_json['annotations']의 image_id가 같음을 이용해서 하나의 이미지에 대해 여러개의 라벨들, annotations을 매칭시킴이 중요.

=> 훈련셋 이미지는 12304장, annotation(bbox 좌표) = 라벨은 101140개

# Dataset-d 다운로드

In [ ]:
!gdown https://drive.google.com/uc?id=1-CB-vkgEtsyt83QFxncDcLui48l4KPLz
!unzip /content/coco_dataset_d.zip
!rm /content/coco_dataset_d.zip

In [ ]:
with open('/content/content/YOLOV5_COCO_format/annotations/instances_train2017.json', 'r') as f:
    train_json = json.load(f)

with open('/content/content/YOLOV5_COCO_format/annotations/instances_val2017.json', 'r') as f:
    val_json = json.load(f)    

# 데이터셋 생성 (json-to-dataframe)

- 데이터프레임을 만들어준 이유 : 모델 훈련에 데이터프레임 형태로 훈련셋을 넣기에 COCO형태의 Json 파일에서 필요한 내용들만 뽑아서 데이터프레임으로 만들어줌.

- 주의할 점 : 단일 이미지에 대해 다중 라벨이므로 하나에 이미지에 대해 각각의 라벨들이 매칭되도록 만들어야함.

In [ ]:
templist = os.listdir('/content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/val2017/') # dataset-b
# templist = os.listdir('/content/content/YOLOV5_COCO_format/val2017') # dataset-d
newlist = []
for i in templist:
  id = i.split('.')[0]
  newlist.append(id+'_val.jpg')

In [ ]:
for i in templist:
  os.rename('/content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/val2017/'+i, '/content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/val2017/'+i.split('.')[0]+'_val.jpg') # dataset-b
#   os.rename('/content/content/YOLOV5_COCO_format/val2017/'+i, '/content/content/YOLOV5_COCO_format/val2017/'+i.split('.')[0]+'_val.jpg') # dataset-d

In [ ]:
%mv /content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/val2017/* /content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/train2017/ # dataset-b
# %mv /content/content/YOLOV5_COCO_format/val2017/* /content/content/YOLOV5_COCO_format/train2017/ # dataset-d

!rm -r /content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/val2017  # dataset-b
# !rm -r /content/content/YOLOV5_COCO_format/val2017 # dataset-d

직접 만든 코드

In [ ]:
# annotations id리스트 생성 

train_animg_id = []
for i in range(len(train_json['annotations'])):
    for j,k in train_json['annotations'][i].items():
        # img_id_list.append(j['image_id'])
        if j == 'image_id':
            # print(k)
            train_animg_id.append(k)
            
val_animg_id = []
for i in range(len(val_json['annotations'])):
    for j,k in val_json['annotations'][i].items():
        # img_id_list.append(j['image_id'])
        if j == 'image_id':
            # print(k)
            val_animg_id.append(k)


# id:image_id 딕셔너리 생성 
train_img_id, val_img_id  = [],[]
train_img_filename, val_img_filename = [], []
for i in range(len(train_json['images'])):
    for j,k in train_json['images'][i].items():
        # print(j,k)
        if j == 'id':
            train_img_id.append(k)
        elif j == 'file_name':
            train_img_filename.append(k)
            

for i in range(len(val_json['images'])):
    for j,k in val_json['images'][i].items():
        # print(j,k)
        if j == 'id':
            val_img_id.append(k)
        elif j == 'file_name':
            name = k.split('.')[0] + '_val.jpg'
            val_img_filename.append(name)
            
train_img = { name:value for name, value in zip(train_img_id, train_img_filename) }
val_img = { name:value for name, value in zip(val_img_id, val_img_filename) }

In [ ]:
train_dic = {'image_id' : [], # = 'file_name'
             'class_id' : [], # = 'category_id'
             'image_path' : [], # file_name에 주소 추가해서 만들어주기 
             'xmin' : [],
             'ymin' : [],
             'xmax' : [],
             'ymax' : []}

val_dic = {  'image_id' : [], # = 'file_name'
             'class_id' : [], # = 'category_id'
             'image_path' : [], # file_name에 주소 추가해서 만들어주기 
             'xmin' : [],
             'ymin' : [],
             'xmax' : [],
             'ymax' : []}


# class_id, bbox 추가 
for i in range(len(train_json['annotations'])):
    for j,k in train_json['annotations'][i].items():
        if j == 'bbox':
            x_min = k[0]- k[2]/2
            if x_min < 0:
                x_min = 0
            y_min = k[1]- k[3]/2
            if y_min < 0:
                y_min = 0
            x_max = k[0]+ k[2]/2
            if x_max < 0:
                x_max = 0
            y_max = k[1]+ k[3]/2
            if y_max<0:
                y_max = 0

            train_dic['xmin'].append(x_min)
            train_dic['ymin'].append(y_min)
            train_dic['xmax'].append(x_max)
            train_dic['ymax'].append(y_max)
        
        elif j == 'category_id':
            train_dic['class_id'].append(k)
        
        # if j == 'image_id':
        #     # print(k)
        #     train_dic['image_id'].append(k)
        
for i in range(len(val_json['annotations'])):
    for j,k in val_json['annotations'][i].items():
        if j == 'bbox':
            x_min = k[0]- k[2]/2
            if x_min < 0:
                x_min = 0
            y_min = k[1]- k[3]/2
            if y_min < 0:
                y_min = 0
            x_max = k[0]+ k[2]/2
            if x_max < 0:
                x_max = 0
            y_max = k[1]+ k[3]/2
            if y_max<0:
                y_max = 0
            
            # min_max = [x_min, y_min, x_max, y_max]
            # val_dic['bbox'].append(min_max)
            
            val_dic['xmin'].append(x_min)
            val_dic['ymin'].append(y_min)
            val_dic['xmax'].append(x_max)
            val_dic['ymax'].append(y_max)

        elif j == 'category_id':
            val_dic['class_id'].append(k)



# image_id, image_path 추가 
train_dir = '/content/content/YOLO2COCO/dataset/YOLOV5_COCO_format/train2017/'


for i in train_animg_id:
    train_dic['image_id'].append(train_img.get(i))
    train_dic['image_path'].append(train_dir + train_img.get(i))   

for i in val_animg_id:
    val_dic['image_id'].append(val_img.get(i))
    val_dic['image_path'].append(train_dir + val_img.get(i))   

     


In [ ]:
train_df = pd.DataFrame(train_dic)
val_df = pd.DataFrame(val_dic)
train_df = pd.concat([train_df, val_df], axis=0).reset_index().drop('index', axis=1)
train_df

,image_id,class_id,image_path,xmin,ymin,xmax,ymax
0,000000000001.jpg,12,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,116.778616,34.730970,186.000000,62.000000
1,000000000001.jpg,1,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,214.095292,90.290528,298.000000,170.000000
2,000000000001.jpg,4,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,46.188259,248.680998,300.000000,320.000000
3,000000000001.jpg,4,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,52.481112,236.388145,300.000000,314.000000
4,000000000001.jpg,1,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,212.095292,88.290528,296.000000,168.000000
...,...,...,...,...,...,...,...
144379,000000005272_val.jpg,4,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,108.315079,60.905434,169.386347,300.145486
144380,000000005272_val.jpg,1,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,246.205551,241.430894,328.402101,315.481386
144381,000000005272_val.jpg,1,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,280.198050,246.141207,346.262566,315.152760
144382,000000005272_val.jpg,4,/content/content/YOLO2COCO/dataset/YOLOV5_COCO...,90.550638,58.166881,168.714179,301.131365


In [ ]:
# 바운딩박스 좌표에 음수 여부 검수용 
for i in range(len(train_json['annotations'])):
    for j,k in train_json['annotations'][i].items():
        if j == 'bbox':
            for box in k:
                if box < 0:
                    print('error')